# Clip routes at district cutoff when displaying them on the GTFS Digest CT District notebooks
* Requested by Evan [here](https://github.com/cal-itp/data-analyses/issues/1386)
* gtfs_digest/district_report.ipynb 

In [9]:
import _report_utils
import _section1_utils as section1
import _section2_utils as section2
import geopandas as gpd
import pandas as pd
from update_vars import GTFS_DATA_DICT, RT_SCHED_GCS
from shared_utils import (
    catalog_utils,
    portfolio_utils,
    rt_dates,
    rt_utils,
    time_helpers,
)

In [10]:
# Data Dictionary
GTFS_DATA_DICT = catalog_utils.get_catalog("gtfs_analytics_data")


In [11]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [12]:
analysis_date_list = [rt_dates.DATES["feb2025"]]

In [13]:
analysis_date = rt_dates.DATES["feb2025"]

In [14]:
schd_vp_url = f"{GTFS_DATA_DICT.digest_tables.dir}{GTFS_DATA_DICT.digest_tables.route_schedule_vp}.parquet"

In [15]:
district = "04 - Oakland"

In [44]:
district_int = [int(s) for s in district.split() if s.isdigit()][0]

In [16]:
# Read in all datasets here with GTFS_DATA_DICT 
# Reran merge_operator_data.py to test this
OPERATOR_FILE = GTFS_DATA_DICT.digest_tables.operator_profiles
OPERATOR_ROUTE = GTFS_DATA_DICT.digest_tables.operator_routes_map

operator_df = pd.read_parquet(
    f"{RT_SCHED_GCS}{OPERATOR_FILE}.parquet",
    filters = [[("caltrans_district", "==", district)]]
)

# using name instead of schedule_gtfs_dataset_key allows us to get 
# the last ones for LA Metro without keeping extraneous rows for LA Metro when keys changed
operator_df = operator_df.sort_values(
    ["service_date", "name"], 
    ascending=[False, True]
).drop_duplicates(
    subset=["name"]
).reset_index(drop=True)

In [17]:
# De duplicate
# First find any organizations_names with more than 2 names per 
orgs_agg = (
    operator_df.groupby(["caltrans_district", "organization_name"])
    .agg({"name": "nunique"})
    .reset_index()
)
orgs_agg2 = orgs_agg.loc[orgs_agg.name > 1]
orgs_with_2_names = list(orgs_agg2.organization_name.unique())

In [18]:
# Delete out these organizations from the original df so we can manipulate them.
operator_df2 = operator_df.loc[
    ~operator_df.organization_name.isin(orgs_with_2_names)
].reset_index(drop=True)

In [19]:
# Filter for these organizations with more than 2 names in their own df.
orgs_with_2_names_df = operator_df.loc[
    operator_df.organization_name.isin(orgs_with_2_names)
].reset_index(drop=True)

In [20]:
three_month_reference = operator_df2["service_date"].max() - pd.DateOffset(
    months=3
)

In [21]:
orgs_with_2_names_df = orgs_with_2_names_df[
    orgs_with_2_names_df["service_date"] >= three_month_reference
]

In [22]:
# Filter out any rows in which `vp_per_min_agency` and `spatial_accuracy_agency` is equal than 0
# and still has 2+ names
orgs_agg = (
    orgs_with_2_names_df.groupby(["organization_name"])
    .agg({"name": "nunique"})
    .reset_index()
    .rename(columns={"name": "n_names"})
)

In [23]:
orgs_with_2_names_df = pd.merge(
    orgs_with_2_names_df, orgs_agg, on="organization_name", how="left"
)

In [24]:
orgs_with_2_names_df2 = orgs_with_2_names_df[
    (orgs_with_2_names_df.vp_per_min_agency > 0)
    & (orgs_with_2_names_df.spatial_accuracy_agency > 0)
    & (orgs_with_2_names_df.n_names > 1)
].reset_index(drop=True)

In [25]:
# Keep rows that meet service_date
service_date = operator_df2.service_date.max()
orgs_with_2_names_df3 = orgs_with_2_names_df2.loc[
    orgs_with_2_names_df2.service_date == service_date
]
final_names = list(orgs_with_2_names_df3.organization_name.unique())

In [26]:
## Concat back
orgs_with_2_names_df = orgs_with_2_names_df.loc[~orgs_with_2_names_df.organization_name.isin(final_names)]

In [27]:
orgs_with_2_names_df_final = pd.concat([orgs_with_2_names_df, orgs_with_2_names_df3])

In [28]:
operator_df2 = pd.concat([operator_df2, orgs_with_2_names_df_final])

In [29]:
OPERATOR_ROUTE = GTFS_DATA_DICT.digest_tables.operator_routes_map

In [31]:
operators_in_district = operator_df2.name.unique()

In [32]:
operator_route_gdf = gpd.read_parquet(
    f"{RT_SCHED_GCS}{OPERATOR_ROUTE}.parquet",
    filters = [["name", "in", operators_in_district]],
    columns = ["name", "service_date", 
               "route_combined_name", "geometry"]
).sort_values(
    ["service_date", "name", "route_combined_name"], 
    ascending=[False, True, True]
).drop_duplicates(
    subset = ["name", "route_combined_name"]
).drop(
    columns = ["service_date", "route_combined_name"]
    # drop route because after the dissolve, all operator routes are combined
    # so route would hold only the first row's value
).dissolve(by = "name").reset_index().pipe(_report_utils.replace_column_names)

In [39]:
operator_route_gdf.columns

Index(['Transit Operator', 'geometry'], dtype='object')

In [55]:
def ct_district(district:int)->gpd.GeoDataFrame:
    """
    Load in Caltrans Shape.
    """
    caltrans_url = "https://gis.data.ca.gov/datasets/0144574f750f4ccc88749004aca6eb0c_0.geojson?outSR=%7B%22latestWkid%22%3A3857%2C%22wkid%22%3A102100%7D"
    ca_geojson = (gpd.read_file(caltrans_url)
               .to_crs(operator_route_gdf.crs))
    district_geojson = ca_geojson.loc[ca_geojson.DISTRICT == district]
    return district_geojson

In [51]:
shapes_within_dist = gpd.sjoin(
            operator_route_gdf,
            district_geojson,
            how = "inner",
            predicate = "within",
        ).drop(columns = "index_right")
    